In [108]:
import pymongo
client=pymongo.MongoClient("mongodb://localhost:27017/")

In [109]:
db=client['test']

In [110]:
collection=db['agent']

In [111]:
ms=collection.find().to_list()

In [113]:
for message in ms:
    message.pop('_id')

In [103]:
from smolagents import TaskStep,ActionStep,PlanningStep

In [8]:
from smolagents.agents import CodeAgent
#知识检索 agent
from smolagents import CodeAgent,OpenAIServerModel,tool,HfApiModel,DuckDuckGoSearchTool
from tools.final_answer import FinalAnswerTool
from tools.visit_webpage import VisitWebpageTool
from tools.vision_comprehension import VisionComprehension
from tools.file_io import file_writer
model=OpenAIServerModel(model_id='qwen-max',api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",api_key="sk-615616fb539749dda57c80cc0928669d")
# model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
agent=CodeAgent(model=model,
                     tools=[DuckDuckGoSearchTool(),file_writer(),VisitWebpageTool()],
                     additional_authorized_imports=['flask','os','matplotlib','pandas','numpy','seaborn','sklearn','torch','transformers','tensorflow','keras','cv2','PIL','matplotlib.pyplot','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL','matplotlib.pyplot as plt','matplotlib.pyplot as plt','pandas as pd','numpy as np','seaborn as sns','sklearn as sk','torch as torch','transformers as transformers','tensorflow as tf','keras as keras','cv2 as cv2','PIL as PIL'],
                     name='qwen_agent',
                     description='information extraction agent,can do rag and web search')


In [14]:
chunks=agent.run(
    "I want to know the weather in beijing,can you tell me the weather?",
    stream=True
)

╭───────────────────────────────────────────── New run - qwen_agent ──────────────────────────────────────────────╮
│                                                                                                                 │
│ I want to know the weather in beijing,can you tell me the weather?                                              │
│                                                                                                                 │
╰─ OpenAIServerModel - qwen-max ──────────────────────────────────────────────────────────────────────────────────╯

In [15]:
result=[]
for chunk in chunks:
    result.append(chunk)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  weather_beijing = web_search(query="current weather in Beijing")                                                 
  print(weather_beijing)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Beijing, Beijing, China Weather Forecast | 
AccuWeather](https://www.accuweather.com/en/cn/beijing/101924/weather-forecast/101924)
Beijing, Beijing, China Weather Forecast, with current conditions, wind, air quality, and what to expect for the 
next 3 days.

[Beijing Weather Forecast Today](https://chinaweather.org/beijing)
Beijing, China Weather. Current: 9.44°C/48.99°F, Wind SW at 8.78 km/h, 33% Humidity, 0% Chance of rain. Beijing, 
China Weather; Current conditions; Weather forecast; Averages and extremes; Yesterday's Data; Temperature and 
possibility of rain in Beijing over the next 12 hours;

[Hourly forecast for Beijing, Beijing Municipality, China - 
timeanddate.com](https://www.timeanddate.com/weather/china/beijing/hourly)
Hour-by-Hour Forecast for Beijing, Beijing Municipality, China. Time/General; Weather . Weather Today/Tomorrow ; 
Hour-by-Hour Forecast ; 14 Day Forecast

[Beijing, Beijing, China Current Weather | 
AccuWeather](https://www.accuweather.com/en/cn/beijing/101924/current-weather/101924)
Current weather in Beijing, Beijing, China. Check current conditions in Beijing, Beijing, China with radar, hourly,
and more.

[Weather in Beijing, Beijing Municipality, China - 
timeanddate.com](https://www.timeanddate.com/weather/china/beijing)
Current weather in Beijing and forecast for today, tomorrow, and next 14 days

[Today's Weather in Beijing - Hourly Forecast and Conditions](https://www.easeweather.com/asia/china/beijing/today)
Stay informed and make the most of your day with our live weather updates from Beijing today. Weather in Beijing 
today The weather in Beijing today is expected to be noticeably warmer than usual, with a forecast temperature of 
21 ° C , compared to an average of 17.5 ° C for 26th of March in recent years.

[Beijing - BBC Weather](https://www.bbc.com/weather/1816670)
14-day weather forecast for Beijing.

[Beijing, People's Republic of China Weather Conditions](https://www.wunderground.com/weather/cn/beijing)
Beijing Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps 
& tropical weather conditions for the Beijing area.

[Weather today - Beijing, China](https://www.weather-atlas.com/en/china/beijing)
Beijing, China - Current temperature and weather conditions. Detailed hourly weather forecast for today - including
weather conditions, temperature, pressure, humidity, precipitation, dewpoint, wind, visibility, and UV index data.

[Weather Beijing - meteoblue](https://www.meteoblue.com/en/weather/week/beijing_china_1816670)
Today's and tonight's professional weather forecast for Beijing. Precipitation radar, HD satellite images, and 
current weather warnings, hourly temperature, chance of rain, and sunshine hours.

Out: None

[Step 1: Duration 4.27 seconds| Input tokens: 2,382 | Output tokens: 43]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The current weather in Beijing is 9.44°C/48.99°F with a wind speed of 8.78 km/h and 33%            
  humidity.")                                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The current weather in Beijing is 9.44°C/48.99°F with a wind speed of 8.78 km/h and 33% 
humidity.

[Step 2: Duration 4.01 seconds| Input tokens: 5,574 | Output tokens: 150]

In [19]:
type(result[0].to_messages())

list

In [21]:
from smolagents import handle_agent_output_types

In [140]:
a=[1,2,3,4,5,6]
b=[2,3,4,5,6,7]


In [164]:
import requests

In [166]:
session=requests.Session()

In [7]:
from flask import Flask,Response
import time

app=Flask(__name__)
@app.route('/events')
def events():
    def generate():
        for i in range(10):
            yield f"data:Message{i}\n\n"
            time.sleep(1)
    return Response(generate(),mimetype='text/event-stream')
if __name__=='__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\aesc\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [33]:
agent.logger.log(result[0])

ActionStep(
    model_input_messages=[
        {
            'role': <MessageRole.SYSTEM: 'system'>,
            'content': [
                {
                    'type': 'text',
                    'text': 'You are an expert assistant who can solve any task using code blobs. You will be given
a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are 
basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a 
series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the 
\'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want
to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end 
with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important 
information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be 
available as input for the next step.\nIn the end you have to return a final answer using the `final_answer` 
tool.\n\nHere are a few examples using notional tools:\n---\nTask: "Generate an image of the oldest person in this 
document."\n\nThought: I will proceed step by step and use the following tools: `document_qa` to find the oldest 
person in the document, then `image_generator` to generate an image according to the answer.\nCode:\n```py\nanswer 
= document_qa(document=document, question="Who is the oldest person 
mentioned?")\nprint(answer)\n```<end_code>\nObservation: "The oldest person in the document is John Doe, a 55 year 
old lumberjack living in Newfoundland."\n\nThought: I will now generate an image showcasing the oldest 
person.\nCode:\n```py\nimage = image_generator("A portrait of John Doe, a 55-year-old man living in 
Canada.")\nfinal_answer(image)\n```<end_code>\n\n---\nTask: "What is the result of the following operation: 5 + 3 +
1294.678?"\n\nThought: I will use python code to compute the result of the operation and then return the final 
answer using the `final_answer` tool\nCode:\n```py\nresult = 5 + 3 + 
1294.678\nfinal_answer(result)\n```<end_code>\n\n---\nTask:\n"Answer the question in the variable `question` about 
the image stored in the variable `image`. The question is in French.\nYou have been provided with these additional 
arguments, that you can access using the keys as variables in your python code:\n{\'question\': \'Quel est 
l\'animal sur l\'image?\', \'image\': \'path/to/image.jpg\'}"\n\nThought: I will use the following tools: 
`translator` to translate the question into English and then `image_qa` to answer the question on the input 
image.\nCode:\n```py\ntranslated_question = translator(question=question, src_lang="French", 
tgt_lang="English")\nprint(f"The translated question is {translated_question}.")\nanswer = image_qa(image=image, 
question=translated_question)\nfinal_answer(f"The answer is {answer}")\n```<end_code>\n\n---\nTask:\nIn a 1979 
interview, Stanislaus Ulam discusses with Martin Sherwin about other great physicists of his time, including 
Oppenheimer.\nWhat does he say was the consequence of Einstein learning too much math on his creativity, in one 
word?\n\nThought: I need to find and read the 1979 interview of Stanislaus Ulam with Martin 
Sherwin.\nCode:\n```py\npages = search(query="1979 interview Stanislaus Ulam Martin Sherwin physicists 
Einstein")\nprint(pages)\n```<end_code>\nObservation:\nNo result found for query "1979 interview Stanislaus Ulam 
Martin Sherwin physicists Einstein".\n\nThought: The query was maybe too restrictive and did not find any results. 
Let\'s try again with a broader query.\nCode:\n```py\npages = search(query="1979 interview Stanislaus 
Ulam")\nprint(pages)\n```<end_code>\nObservation:\nFound 6 pages:\n[Stanislaus Ulam 1979 
interview](https://ahf.nuclearmuseum.org/voic

In [29]:
agent.monitor.step_durations

[4.27470326423645, 4.0052330493927]